In [45]:
import numpy as np
import os
from tqdm import tqdm
from bs4 import BeautifulSoup
import re
cleanr = re.compile('<.*?>')
replace_list=['\r','\n','\t','&quot','On']
def cleanhtml(line):
    for i in replace_list:
        line = line.replace(i, '')

    cleantext = re.sub(cleanr, '',line)
    
    
    return cleantext

In [46]:
label=[]
text=[]
for i in tqdm(os.listdir("scraped_Set/")):
    if i=="3.html":
        continue
    dict1=np.load("scraped_Set/"+i,allow_pickle=True).item()
    articles=dict1['results']
    for art in articles:
        label.append(art["ruling_slug"])
        text.append(cleanhtml(art["statement"]))
    
    

100%|█████████████████████████████████████████████████████████████████████████████| 1780/1780 [00:02<00:00, 755.94it/s]


In [47]:

import pandas as pd
df=pd.DataFrame()

In [48]:
df['label']=label
df['text']=text

In [49]:
np.unique(df['label'],return_counts=True)

(array(['barely-true', 'false', 'full-flop', 'half-flip', 'half-true',
        'mostly-true', 'no-flip', 'pants-fire', 'true'], dtype=object),
 array([2931, 3731,  162,   71, 3289, 3116,   27, 2047, 2407], dtype=int64))

In [50]:
false_list=[]
neutral_list=[]           
true_list=[]

In [51]:
mapping={'barely-true':'false','false':'false','pants-fire':'false','half-true':'neutral','mostly-true':'true','true':'true'}

In [52]:
df['label']=df['label'].map(mapping)

In [53]:
df=df[df['label'].isin(['true','false','neutral'])]

In [54]:
np.unique(df['label'],return_counts="True")

(array(['false', 'neutral', 'true'], dtype=object),
 array([8709, 3289, 5523], dtype=int64))

In [63]:
validation=["Novel coronavirus","SARS-CoV-2","covid","cov","corona","virus","COVID-19","Nonmedical","masks","ppe","medical","face coverings",
"N95","Ventilators","novel","wuhan","BiPap","SARS","MERS","respiratory","illness","disease","positive","korona"]

In [68]:
tru_list=[False]*len(df)
count=0
for k,i in enumerate(df['text'].values):
    i=i.lower()
    for j in validation:
        j=j.lower()
        if j in i:
            count=count+1
            tru_list[k]=True
            break

In [70]:
np.unique(df[tru_list]['label'],return_counts=True)

(array(['false', 'neutral', 'true'], dtype=object),
 array([601, 185, 291], dtype=int64))

In [71]:
df1=df[tru_list]

In [72]:
df1.to_csv("fake_covid.csv")

In [73]:
articles[0]

{'id': 8263,
 'slug': 'van-jones-says-un-has-conducted-172-successful-pea',
 'speaker': {'slug': 'van-jones',
  'full_name': 'Van Jones',
  'first_name': 'Van',
  'last_name': 'Jones'},
 'targets': [],
 'statement': '<p>The United Nations has conducted &quot;172 successful peacekeeping missions and negotiations.&quot;</p>\r\n',
 'ruling_slug': 'mostly-true',
 'publication_date': '2013-09-12T15:56:56-04:00',
 'ruling_comments': '<p>With the U.S. and Russia locked in discussions over whether, and how, to exert control over Syria&rsquo;s chemical weapons stockpiles, the United Nations&rsquo; role in international arms control is of particular interest.</p>\r\n\r\n<p>During a discussion on <a href="http://transcripts.cnn.com/TRANSCRIPTS/1309/10/cfr.01.html"><u>CNN&rsquo;s </u><em><u>Crossfire</u></em></a>, host Van Jones pointed to the United Nations&rsquo; long record of successful actions to keep the peace and protect civilians.</p>\r\n\r\n<p>When co-host S.E. Cupp asked, &quot;Do you tr